In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# MySQL 접속 설정
username = "root"      # MySQL 사용자명
password = "1234"      # MySQL 비밀번호
host = "127.0.0.1"     # 로컬호스트
port = 3306            # 포트
database = "ott_db"  # DB 이름

# MySQL 엔진 생성
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [3]:
# 데이터 파일 경로 설정
folder_path = r"방송통신위원회_해외 OTT 이용행태조사 원시데이터_20231231"
file_2022 = os.path.join(folder_path, "2022해외OTT이용행태조사_국가통합(20240403).csv")
file_2023 = os.path.join(folder_path, "2023해외OTT이용행태조사_국가통합(20240403).csv")

# CSV 읽기
df_2022 = pd.read_csv(file_2022, encoding='utf-8-sig', low_memory=False)
df_2023 = pd.read_csv(file_2023, encoding='utf-8-sig', low_memory=False)

print("2022 데이터:", df_2022.shape)
print("2023 데이터:", df_2023.shape)

2022 데이터: (4536, 1903)
2023 데이터: (6326, 929)


## 2. device
### (1) user_id

In [4]:
# 1. 2022 user_id 생성
df_2022['user_id'] = ('2022' + df_2022['RESPID'].astype(str)).astype(int)
# 2. 2023 user_id 생성
df_2023['user_id'] = ('2023' + df_2023['id'].astype(str)).astype(int)

df_user_id_2022 = df_2022[['user_id']].copy()
df_user_id_2023 = df_2023[['user_id']].copy()

# 3. 합치기
df_user_id = pd.concat([df_user_id_2022, df_user_id_2023], ignore_index=True)

# 4. device 테이블에 user_id 업로드
df_user_id.to_sql(name="device", con=engine, index=False, if_exists="replace")

10862

In [5]:
def add_column_to_device(df_device, df_2022, df_2023, col_2022, col_2023, new_col_name,
                       mapping_2022=None, mapping_2023=None, default_value='사용안함'):
    """
    device 테이블에 새로운 컬럼을 추가하는 함수
    """
    # 2022 데이터 처리
    df_2022_col = df_2022[['user_id', col_2022]].copy()
    df_2022_col = df_2022_col.rename(columns={col_2022: new_col_name})
    df_2022_col[new_col_name] = pd.to_numeric(df_2022_col[new_col_name], errors='coerce')  # 숫자 변환
    if mapping_2022:
        df_2022_col[new_col_name] = df_2022_col[new_col_name].map(mapping_2022).fillna(default_value)
    else:
        df_2022_col[new_col_name] = df_2022_col[new_col_name].fillna(default_value)

    # 2023 데이터 처리
    df_2023_col = df_2023[['user_id', col_2023]].copy()
    df_2023_col = df_2023_col.rename(columns={col_2023: new_col_name})
    df_2023_col[new_col_name] = pd.to_numeric(df_2023_col[new_col_name], errors='coerce')  # 숫자 변환
    if mapping_2023:
        df_2023_col[new_col_name] = df_2023_col[new_col_name].map(mapping_2023).fillna(default_value)
    else:
        df_2023_col[new_col_name] = df_2023_col[new_col_name].fillna(default_value)

    # 합치기 (2023 우선)
    df_new_col = pd.concat([df_2022_col, df_2023_col], ignore_index=True)
    df_new_col = df_new_col.drop_duplicates('user_id', keep='last')  # 2023 값 우선

    # device 테이블 병합
    df_device = df_device.merge(df_new_col, on='user_id', how='left')

    return df_device

In [6]:
# 현재 device 테이블 불러오기
df_device = pd.read_sql("SELECT * FROM device", con=engine)

### (2) smartphone_frequency

In [7]:
frequency_map = {
    1: '매일',
    2: '1주일에 5~6일',
    3: '1주일에 3~4일',
    4: '1주일에 1~2일',
    5: '한 달에 1~3일',
    6: '2~3달에 1~2일 이하'
}

df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A1_1", col_2023="G1",
    new_col_name="smartphone_frequency",
    mapping_2022=frequency_map,
    mapping_2023=frequency_map
)

### (3) tablet_frequency

In [8]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A1_2", col_2023="G1_n2",
    new_col_name="tablet_frequency",
    mapping_2022=frequency_map,
    mapping_2023=frequency_map
)

### (4) pc_frequency

In [9]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A1_3", col_2023="G1_n3",
    new_col_name="pc_frequency",
    mapping_2022=frequency_map,
    mapping_2023=frequency_map
)

### (5) tv_frequency

In [12]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A1_4", col_2023="G1_n4",
    new_col_name="tv_frequency",
    mapping_2022=frequency_map,
    mapping_2023=frequency_map
)

### (6) console_frequency

In [14]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A1_5", col_2023="G1_n5",
    new_col_name="console_frequency",
    mapping_2022=frequency_map,
    mapping_2023=frequency_map
)

### (7) smartphone_hours_weekdays

In [16]:
hours_map ={
    1:	'30분 미만',
    2:	'30분~1시간 미만',
    3:	'1시간~1시간 30분 미만',
    4:	'1시간~2시간 미만',
    5:	'2시간~3시간 미만',
    6:	'3시간~4시간 미만',
    7:	'4시간~5시간 미만',
    8:	'5시간~6시간',
    9:	'6시간 이상'
}

df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_1_3", col_2023="G2_1",
    new_col_name="smartphone_hours_weekdays",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (8) smartphone_hours_weekend

In [18]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_1_4", col_2023="G2_2",
    new_col_name="smartphone_hours_weekend",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (9) tablet_hours_weekdays

In [20]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_2_3", col_2023="G2_1_n2",
    new_col_name="tablet_hours_weekdays",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (10) tablet_hours_weekend

In [22]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_2_4", col_2023="G2_2_n2",
    new_col_name="tablet_hours_weekend",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (11) pc_hours_weekdays

In [24]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_3_3", col_2023="G2_1_n3",
    new_col_name="pc_hours_weekdays",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (12) pc_hours_weekend

In [26]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_3_4", col_2023="G2_2_n3",
    new_col_name="pc_hours_weekend",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (13) tv_hours_weekdays

In [27]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_4_3", col_2023="G2_1_n4",
    new_col_name="tv_hours_weekdays",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (14) tv_hours_weekend

In [29]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_4_4", col_2023="G2_2_n4",
    new_col_name="tv_hours_weekend",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (15) console_hours_weekdays

In [31]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_5_3", col_2023="G2_1_n5",
    new_col_name="console_hours_weekdays",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (16) console_hours_weekend

In [32]:
df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="A2_5_4", col_2023="G2_2_n5",
    new_col_name="console_hours_weekend",
    mapping_2022=hours_map,
    mapping_2023=hours_map
)

### (17) ott_main_device

In [38]:
ott_device_map = {
    1:	'스마트폰',
    2:	'태블릿 컴퓨터',
    3:	'데스크탑PC/노트북',
    4:	'TV수상기',
    5:	'기타'
}

df_device = add_column_to_device(
    df_device, df_2022, df_2023,
    col_2022="B9", col_2023="A7",
    new_col_name="ott_main_device",
    mapping_2022=ott_device_map,
    mapping_2023=ott_device_map
)

In [39]:
df_device

,user_id,smartphone_frequency,tablet_frequency,pc_frequency,tv_frequency,console_frequency,smartphone_hours_weekdays,smartphone_hours_weekend,tablet_hours_weekdays,tablet_hours_weekend,pc_hours_weekdays,pc_hours_weekend,tv_hours_weekdays,tv_hours_weekend,console_hours_weekdays,console_hours_weekend,ott_main_device
0,20228,매일,매일,한 달에 1~3일,사용안함,사용안함,30분 미만,30분 미만,30분 미만,30분 미만,30분 미만,30분 미만,사용안함,사용안함,사용안함,사용안함,TV수상기
1,202211,매일,사용안함,사용안함,사용안함,사용안함,3시간~4시간 미만,6시간 이상,사용안함,사용안함,사용안함,사용안함,사용안함,사용안함,사용안함,사용안함,스마트폰
2,202212,매일,1주일에 1~2일,1주일에 5~6일,매일,1주일에 1~2일,1시간~1시간 30분 미만,30분~1시간 미만,30분 미만,30분 미만,30분 미만,30분 미만,3시간~4시간 미만,3시간~4시간 미만,30분 미만,30분 미만,스마트폰
3,202213,사용안함,매일,사용안함,사용안함,사용안함,사용안함,사용안함,1시간~2시간 미만,5시간~6시간,사용안함,사용안함,사용안함,사용안함,사용안함,사용안함,TV수상기
4,202215,1주일에 1~2일,1주일에 3~4일,한 달에 1~3일,1주일에 1~2일,1주일에 3~4일,1시간~2시간 미만,2시간~3시간 미만,1시간~1시간 30분 미만,1시간~1시간 30분 미만,4시간~5시간 미만,2시간~3시간 미만,3시간~4시간 미만,1시간~2시간 미만,1시간~2시간 미만,2시간~3시간 미만,데스크탑PC/노트북
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10857,202310298,1주일에 5~6일,1주일에 5~6일,매일,매일,사용안함,1시간~2시간 미만,1시간~1시간 30분 미만,1시간~2시간 미만,1시간~1시간 30분 미만,1시간~2시간 미만,30분~1시간 미만,1시간~1시간 30분 미만,1시간~1시간 30분 미만,사용안함,사용안함,스마트폰
10858,202310299,매일,사용안함,매일,1주일에 5~6일,사용안함,30분~1시간 미만,30분~1시간 미만,사용안함,사용안함,30분~1시간 미만,30분~1시간 미만,30분~1시간 미만,30분~1시간 미만,사용안함,사용안함,TV수상기
10859,202310300,1주일에 5~6일,1주일에 3~4일,1주일에 3~4일,1주일에 3~4일,1주일에 5~6일,1시간~2시간 미만,1시간~1시간 30분 미만,1시간~1시간 30분 미만,30분~1시간 미만,30분~1시간 미만,30분~1시간 미만,30분~1시간 미만,30분~1시간 미만,1시간~1시간 30분 미만,30분~1시간 미만,스마트폰
10860,202310301,매일,사용안함,매일,매일,사용안함,1시간~2시간 미만,30분~1시간 미만,사용안함,사용안함,2시간~3시간 미만,1시간~1시간 30분 미만,2시간~3시간 미만,1시간~2시간 미만,사용안함,사용안함,TV수상기


In [40]:
# 최종 DB 업로드
df_device.to_sql(name="device", con=engine, index=False, if_exists="replace")
print("device 테이블 최종 업데이트 완료!")

device 테이블 최종 업데이트 완료!
